In [ ]:
# 1. Setup & Android Engine
print("⏳ [1/3] Starting Android Engine...")
!apt-get update -qq && apt-get install -y docker.io adb -qq > /dev/null
!docker run -d --privileged -p 5555:5555 redroid/redroid:11.0.0-latest > /dev/null

# 2. Launch Viewer (The Web Interface)
print("🌐 [2/3] Launching Web Viewer...")
!npx -y ws-scrcpy-docker --port 7860 > /dev/null 2>&1 &

# 3. Install & Start Cloudflare Tunnel
print("🔗 [3/3] Creating Secure Link... (Wait for the URL to appear)")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb > /dev/null

import os
import subprocess
import threading
import time

def run_tunnel():
    # This runs the tunnel and prints the logs so you can see the URL
    subprocess.run(["cloudflared", "tunnel", "--url", "http://localhost:7860"])

threading.Thread(target=run_tunnel, daemon=True).start()

# Give it a moment to generate the link
time.sleep(10)
print("\n✅ SUCCESS! Look for the link ending in '.trycloudflare.com' below:")